In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
experiment_id = 'klaus_2018-09-24'

In [ ]:
experiment_base_path = Path('/home/klaus/dev/revolve-simulator/l-system/experiments/')
experiment_path = experiment_base_path / experiment_id
if not experiment_path.exists():
    raise Exception(f'Error. Experiment {str(experiment_path)} does not exist.')

In [ ]:
df = pd.read_csv(experiment_path / 'measures2.txt', sep= ' ')
df.head()

In [ ]:
df = pd.pivot_table(df, values='value', index=['generation', 'genome'], columns=['measures'], aggfunc='last')
df.head()